In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory. 
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
EMBEDDING_FILE = 'adfasfad'#'/home/kai/data/resources/glove/glove.840B.300d.txt' #glove.twitter.27B.200d.txt
PATH = '~/data/toxic/data/'
# train = pd.read_csv(PATH + 'train.csv')
# test = pd.read_csv(PATH + 'test.csv')
# train = pd.read_csv(PATH + 'cleaned_train.csv')
# test = pd.read_csv(PATH + 'cleaned_test.csv')
train = pd.read_csv(PATH + 'train_preprocessed.csv')
test = pd.read_csv(PATH + 'test_preprocessed.csv')

In [3]:
train.head(1)

,comment_text,id,identity_hate,insult,obscene,set,severe_toxic,threat,toxic,toxicity
0,explanation why the edits made under my userna...,0000997932d777bf,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0


In [4]:
def word2Vec(source):
    embed_size = 300
    if source.lower() == 'ft-common':
        file = '/home/kai/data/resources/FastText/crawl-300d-2M.vec'
    elif source.lower() == 'ft-wiki':
        file = '/home/kai/data/resources/FastText/wiki.en.vec'
    elif source.lower() == 'lex':
        file = '/home/kai/data/resources/lexvec/lexvec.commoncrawl.300d.W.pos.vectors'
    elif source.lower() == 'gl-common':
        file = '/home/kai/data/resources/glove/glove.840B.300d.txt'
    elif source.lower() == 'gl-twitter':
        file = '/home/kai/data/resources/glove/glove.twitter.27B.200d.txt'
        embed_size = 200
    def get_coefs(word,*arr): 
        try:
            return word, np.asarray(arr, dtype='float32') 
        except ValueError:
            return 'nnnnnnnaaaaaaa@@!',np.zeros(embed_size)
    embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(file, encoding='utf8'))
    return embeddings_index, embed_size

In [5]:
embeddings_index_lex, embed_size = word2Vec('lex')
embeddings_index_glc, embed_size = word2Vec('gl-common')
embeddings_index_glt, embed_size = word2Vec('gl-twitter')
embeddings_index_ftc, embed_size = word2Vec('ft-common')
embeddings_index_ftw, embed_size = word2Vec('ft-wiki')

In [9]:
comment_col = 'comment_text' # 'comment_text_cleaned' 

X_train = train[comment_col].str.lower().fillna('something') # something is a word of neutral sentiment
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

X_test = test[comment_col].str.lower().fillna('something')

In [11]:
max_features=100000
maxlen=150

tok=text.Tokenizer(num_words=max_features,lower=True)
tok.fit_on_texts(list(X_train)+list(X_test))
X_train=tok.texts_to_sequences(X_train)
X_test=tok.texts_to_sequences(X_test)
x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
x_test=sequence.pad_sequences(X_test,maxlen=maxlen)

In [33]:
sorted_word_count = sorted(tok.word_counts.items(), key=lambda x: x[1], reverse=True)

In [65]:
#sorted_word_count#[60000:]

In [59]:
# count = 0
# for w,w_cnt in sorted_word_count[:60000]:
#     if w in embeddings_index_glt:
#         count+=1
# print(count)
# print(count/60000)

48862
0.8143666666666667


In [63]:
# type(embeddings_index_glt['hesitates'])

numpy.ndarray

In [66]:
embeddings_index = embeddings_index_lex
word_index = tok.word_index
#prepare embedding matrix
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
not_found_word = {}
for word, i in word_index.items():
    if i >= max_features:
        continue
    try: 
        embedding_vector = embeddings_index[word] # w2v_model['/en/'+ word] #w2v_model[word]#
    except KeyError:
        embedding_vector = embeddings_index.get(bad_word_dict.get(word), None)
#         embedding_vector = None #np.zeros(embed_size)
        if embedding_vector is None:
            not_found_word[word] = tok.word_counts[word]#i
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print(len(not_found_word))

9787


In [8]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [15]:
embeddings_index_pool = [
    (embeddings_index_lex, 300, 'lex'),
    (embeddings_index_glc, 300, 'glc'),
    (embeddings_index_glt, 200, 'glt'),
    (embeddings_index_ftc, 300, 'ftc'), 
    (embeddings_index_ftw, 300, 'ftw')
]

In [26]:
batch_epoch_patience_pool = [
    (1024, 10, 10),
    (1024, 3, 3),
    (1024, 3, 3),
    (1024, 3, 3),
    (512, 8, 8),
    (512, 2, 2),
    (512, 2, 2),
    (512, 2, 2),
    (128, 5, 5),
    (128, 2, 2),
    (128, 2, 2),
    (128, 1, 1),
    (32, 3, 3),
    (32, 2, 2),
    (32, 1, 1),
    (32, 1, 1),
]

In [ ]:
for embeddings_index, embed_size, embedding_name in embeddings_index_pool:
    
    word_index = tok.word_index
    #prepare embedding matrix
    num_words = min(max_features, len(word_index) + 1)
    embedding_matrix = np.zeros((num_words, embed_size))
    not_found_word = {}
    for word, i in word_index.items():
        if i >= max_features:
            continue
        try: 
            embedding_vector = embeddings_index[word] # w2v_model['/en/'+ word] #w2v_model[word]#
        except KeyError:
            embedding_vector = embeddings_index.get(bad_word_dict.get(word), None)
    #         embedding_vector = None #np.zeros(embed_size)
            if embedding_vector is None:
                not_found_word[word] = tok.word_counts[word]#i
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    print(len(not_found_word))
    
    for batch_size, epochs, patience in batch_epoch_patience_pool:
        print(batch_size, epochs, patience)
    
        X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.9)#, random_state=233)

        sequence_input = Input(shape=(maxlen, ))
        x = Embedding(max_features, embed_size, weights=[embedding_matrix],trainable = False)(sequence_input)
        x = SpatialDropout1D(0.2)(x)
        x = Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
        x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
        avg_pool = GlobalAveragePooling1D()(x) 
        max_pool = GlobalMaxPooling1D()(x)
        x = concatenate([avg_pool, max_pool]) 
        # x = Dense(128, activation='relu')(x)
        # x = Dropout(0.1)(x)
        preds = Dense(6, activation="sigmoid")(x)
        model = Model(sequence_input, preds)



        model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-3),metrics=['accuracy'])

        run_name = "rnn_w_conv_{}_{}_w_dict".format(embedding_name, batch_size)
        print(run_name)

        filepath= './NewRnnModels/' + run_name + '.hdf5'
        
        try: 
            print('load model: ' + str(filepath))
            model.load_weights(filepath)
        except:
            print('no model found')
        
        checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
        early = EarlyStopping(monitor="val_acc", mode="max", patience=patience)
        ra_val = RocAucEvaluation(validation_data=(X_val, y_val), interval = 1)
        callbacks_list = [ra_val,checkpoint, early]

        model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),callbacks = callbacks_list,verbose=1)
        #Loading model weights
        model.load_weights(filepath)
        print('Predicting using the best model/epoch so far....')
        y_pred = model.predict(x_test,batch_size=1024,verbose=1)

        y_pred.shape

        submission = pd.read_csv(PATH + 'sample_submission.csv')
        submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
        import time
        sub_id = int(time.time())
        print(sub_id)
        submission.to_csv('./NewRnnPreds/' + run_name + '_' + str(sub_id) + '.csv', index=False)

9787
1024 10 10


/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


rnn_w_conv_lex_1024_w_dict
load model: ./NewRnnModels/rnn_w_conv_lex_1024_w_dict.hdf5
no model found
Train on 143613 samples, validate on 15958 samples
Epoch 1/10
143360/143613 [============================>.] - ETA: 0s - loss: 0.1128 - acc: 0.9658
 ROC-AUC - epoch: 1 - score: 0.956641

Epoch 00001: val_acc improved from -inf to 0.98068, saving model to ./NewRnnModels/rnn_w_conv_lex_1024_w_dict.hdf5
143613/143613 [==============================] - 78s 541us/step - loss: 0.1126 - acc: 0.9658 - val_loss: 0.0533 - val_acc: 0.9807
Epoch 2/10
143360/143613 [============================>.] - ETA: 0s - loss: 0.0518 - acc: 0.9811
 ROC-AUC - epoch: 2 - score: 0.979269

Epoch 00002: val_acc improved from 0.98068 to 0.98231, saving model to ./NewRnnModels/rnn_w_conv_lex_1024_w_dict.hdf5
143613/143613 [==============================] - 71s 495us/step - loss: 0.0518 - acc: 0.9811 - val_loss: 0.0469 - val_acc: 0.9823
Epoch 3/10
143360/143613 [============================>.] - ETA: 0s - loss: 0.0476 

143613/143613 [==============================] - 119s 827us/step - loss: 0.0844 - acc: 0.9730 - val_loss: 0.0484 - val_acc: 0.9821
Epoch 2/8
143360/143613 [============================>.] - ETA: 0s - loss: 0.0484 - acc: 0.9818
 ROC-AUC - epoch: 2 - score: 0.979491

Epoch 00002: val_acc improved from 0.98213 to 0.98296, saving model to ./NewRnnModels/rnn_w_conv_lex_512_w_dict.hdf5
143613/143613 [==============================] - 111s 771us/step - loss: 0.0484 - acc: 0.9818 - val_loss: 0.0446 - val_acc: 0.9830
Epoch 3/8
143360/143613 [============================>.] - ETA: 0s - loss: 0.0453 - acc: 0.9825
 ROC-AUC - epoch: 3 - score: 0.983550

Epoch 00003: val_acc improved from 0.98296 to 0.98351, saving model to ./NewRnnModels/rnn_w_conv_lex_512_w_dict.hdf5
143613/143613 [==============================] - 109s 759us/step - loss: 0.0453 - acc: 0.9825 - val_loss: 0.0429 - val_acc: 0.9835
Epoch 4/8
143360/143613 [============================>.] - ETA: 0s - loss: 0.0430 - acc: 0.9834
 ROC-AU

143488/143613 [============================>.] - ETA: 0s - loss: 0.0388 - acc: 0.9848
 ROC-AUC - epoch: 2 - score: 0.990203

Epoch 00002: val_acc improved from 0.98497 to 0.98516, saving model to ./NewRnnModels/rnn_w_conv_lex_128_w_dict.hdf5
143613/143613 [==============================] - 343s 2ms/step - loss: 0.0388 - acc: 0.9848 - val_loss: 0.0383 - val_acc: 0.9852
Predicting using the best model/epoch so far....
153164/153164 [==============================] - 12s 78us/step
1521304049
128 2 2
rnn_w_conv_lex_128_w_dict
load model: ./NewRnnModels/rnn_w_conv_lex_128_w_dict.hdf5
Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143488/143613 [============================>.] - ETA: 0s - loss: 0.0373 - acc: 0.9853
 ROC-AUC - epoch: 1 - score: 0.991808

Epoch 00001: val_acc improved from -inf to 0.98628, saving model to ./NewRnnModels/rnn_w_conv_lex_128_w_dict.hdf5
143613/143613 [==============================] - 355s 2ms/step - loss: 0.0373 - acc: 0.9853 - val_loss: 0.0358 - v

143360/143613 [============================>.] - ETA: 0s - loss: 0.0380 - acc: 0.9849
 ROC-AUC - epoch: 7 - score: 0.988709

Epoch 00007: val_acc improved from 0.98476 to 0.98477, saving model to ./NewRnnModels/rnn_w_conv_glc_1024_w_dict.hdf5
143613/143613 [==============================] - 71s 493us/step - loss: 0.0380 - acc: 0.9849 - val_loss: 0.0385 - val_acc: 0.9848
Epoch 8/10
143360/143613 [============================>.] - ETA: 0s - loss: 0.0369 - acc: 0.9853
 ROC-AUC - epoch: 8 - score: 0.988945

Epoch 00008: val_acc did not improve
143613/143613 [==============================] - 71s 495us/step - loss: 0.0369 - acc: 0.9853 - val_loss: 0.0386 - val_acc: 0.9847
Epoch 9/10
143360/143613 [============================>.] - ETA: 0s - loss: 0.0361 - acc: 0.9856
 ROC-AUC - epoch: 9 - score: 0.989029

Epoch 00009: val_acc did not improve
143613/143613 [==============================] - 72s 504us/step - loss: 0.0361 - acc: 0.9856 - val_loss: 0.0431 - val_acc: 0.9828
Epoch 10/10
143360/14

153164/153164 [==============================] - 11s 73us/step
1521316628
512 2 2
rnn_w_conv_glc_512_w_dict
load model: ./NewRnnModels/rnn_w_conv_glc_512_w_dict.hdf5
Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143360/143613 [============================>.] - ETA: 0s - loss: 0.0358 - acc: 0.9858
 ROC-AUC - epoch: 1 - score: 0.993488

Epoch 00001: val_acc improved from -inf to 0.98654, saving model to ./NewRnnModels/rnn_w_conv_glc_512_w_dict.hdf5
143613/143613 [==============================] - 124s 862us/step - loss: 0.0358 - acc: 0.9858 - val_loss: 0.0328 - val_acc: 0.9865
Epoch 2/2
143360/143613 [============================>.] - ETA: 0s - loss: 0.0341 - acc: 0.9864
 ROC-AUC - epoch: 2 - score: 0.993185

Epoch 00002: val_acc did not improve
143613/143613 [==============================] - 111s 774us/step - loss: 0.0341 - acc: 0.9864 - val_loss: 0.0336 - val_acc: 0.9863
Predicting using the best model/epoch so far....
153164/153164 [==============================] - 11

Epoch 2/3
143584/143613 [============================>.] - ETA: 0s - loss: 0.0422 - acc: 0.9836
 ROC-AUC - epoch: 2 - score: 0.990148

Epoch 00002: val_acc improved from 0.98356 to 0.98399, saving model to ./NewRnnModels/rnn_w_conv_glc_32_w_dict.hdf5
143613/143613 [==============================] - 1269s 9ms/step - loss: 0.0422 - acc: 0.9836 - val_loss: 0.0404 - val_acc: 0.9840
Epoch 3/3
143584/143613 [============================>.] - ETA: 0s - loss: 0.0394 - acc: 0.9845
 ROC-AUC - epoch: 3 - score: 0.989710

Epoch 00003: val_acc did not improve
143613/143613 [==============================] - 1270s 9ms/step - loss: 0.0394 - acc: 0.9845 - val_loss: 0.0459 - val_acc: 0.9818
Predicting using the best model/epoch so far....
153164/153164 [==============================] - 11s 72us/step
1521324887
32 2 2
rnn_w_conv_glc_32_w_dict
load model: ./NewRnnModels/rnn_w_conv_glc_32_w_dict.hdf5
Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143584/143613 [============================>

In [43]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 150, 300)     30000000    input_5[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_5 (SpatialDro (None, 150, 300)     0           embedding_5[0][0]                
__________________________________________________________________________________________________
bidirectional_5 (Bidirectional) (None, 150, 256)     329472      spatial_dropout1d_5[0][0]        
__________________________________________________________________________________________________
conv1d_5 (

In [18]:
bad_word_dict = {
    "'ass": 'ass',
     "'shit": 'shit',
     "'stupid": 'stupid',
     'asse': 'ass',
     'asspie': 'ass',
     "bitch'": 'bitch',
     'bitchbot': 'bitch',
     'bith': 'bitch',
     'bithc': 'bitch',
     'bithces': 'bitch',
     'choked': 'choke',
     'cocain': 'cocaine',
     'cuck': 'cock',
     'cucks': 'cocks',
     'decease': 'deceased',
     'deth': 'death',
     'diedres': 'crap',
     'dik': 'dick',
     'donkeysex': 'dick',
     'faggt': 'faggot',
     "fool'": 'fool',
     "fuck'": 'fuck',
     'fuckerucker': 'fucker',
     'fuckn': 'fuck',
     'fuock': 'fuck',
     'gayy': 'gay',
     'headsdick': 'dick',
     'homopetersymonds': 'homo',
     'horsecock': 'cock',
     'maoth': 'mouth',
     "mother's": 'mother',
     'motherfuc': 'motherfucker',
     'mothjer': 'mother',
     'niggetr': 'nigger',
     'niggors': 'nigger',
     'nonense': 'nonsense',
     'nonsesnse': 'nonsense',
     'peenus': 'penis',
     'peni': 'penis',
     "penis'": 'penis',
     'pneis': 'penis',
     'pornn': 'porn',
     'semite': 'semitic',
     'sexe': 'sex',
     'sexsex': 'sex',
     'sexualit': 'sexuality',
     'sexuall': 'sexual',
     'sockpuppet': 'alias',
     'sockpuppetry': 'alias',
     'vagpenis': 'penis',
     'valentin': 'valentine',
     'youfuck': 'fuck',
     'zdick': 'dick'
}